In [456]:
import gspread
import torch
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [457]:
credentials_path = '../riasec-model-1-c1119188e695.json' 

spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1x4e-yyXWHMZW9ZXiOb_oUnZLHukOWg1cc0Qf5jL2aLM/edit?usp=sharing'

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
gc = gspread.authorize(credentials)

spreadsheet = gc.open_by_url(spreadsheet_url)

worksheet = spreadsheet.get_worksheet(0)

data = worksheet.get_all_values()

df = pd.DataFrame(data[1:], columns=data[0])

In [458]:
df

,Позначка часу,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",Чи вам подобається займатися фізичною активністю та спортом?,Чи цікавить вас робота з різноманітними інструментами?,Чи вам подобається працювати на відкритому повітрі?,Чи вам цікаво розв'язувати складні математичні задачі?,Чи любите ви читати наукові книги або журнали?,Мені завжди цікаво почути унікальні точки зору на речі,...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [459]:
df.columns

Index(['Позначка часу',
       'Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне',
       'Моя енергія ніколи не вичерпується',
       'Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)',
       'Чи вам подобається займатися фізичною активністю та спортом? ',
       'Чи цікавить вас робота з різноманітними інструментами? ',
       'Чи вам подобається працювати на відкритому повітрі? ',
       'Чи вам цікаво розв'язувати складні математичні задачі? ',
       'Чи любите ви читати наукові книги або журнали? ',
       'Мені завжди цікаво почути унікальні точки зору на речі',
       'Мені подобається вивчати різноманітну інформацію про будь-яку тему',
       'Як ви відноситесь до вивчення нових технологій та програм? ',
       'Люди часто кажуть мені, що мої ідеї геніальні',
       'Немає такої форми мистецтва чи події, яку б я не хотів би дослідити/відвідати',
       'Мені важливий про

In [460]:
element_type = type(df["Чи вам подобається займатися фізичною активністю та спортом? "][0])

print(element_type)

<class 'str'>


In [461]:
numeric_columns = df.columns[1:]

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df

,Позначка часу,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",Чи вам подобається займатися фізичною активністю та спортом?,Чи цікавить вас робота з різноманітними інструментами?,Чи вам подобається працювати на відкритому повітрі?,Чи вам цікаво розв'язувати складні математичні задачі?,Чи любите ви читати наукові книги або журнали?,Мені завжди цікаво почути унікальні точки зору на речі,...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,3,2,4,3,5,4,3,5,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,2,3,1,4,3,5,3,4,2,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,3,4,5,5,4,4,3,5,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,1,1,5,1,1,5,1,1,1,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5,5,3,4,4,5,5,5,5,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,1,1,1,4,5,5,3,1,5,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,5,4,5,5,4,4,4,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,5,2,3,4,3,2,3,3,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,4,3,2,5,4,4,4,4,5,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4,2,4,3,2,4,2,3,2,...,2,3,4,3,4,3,4,3,4,3


In [462]:
element_type = type(df["Чи вам подобається займатися фізичною активністю та спортом? "][0])

print(element_type)

<class 'numpy.int64'>


In [463]:

numeric_columns = df.columns[1:]
riasec_columns = ['Realistic', 'Investigative', 'Artistic', 'Social', 'Enterprising', 'Conventional']
number_of_questions = 36
questions_per_section = 6
averages_df = pd.DataFrame()

for i in range(0, number_of_questions, questions_per_section):
    start_idx = i
    end_idx = i + questions_per_section
    selected_cols = df.columns[start_idx+1:end_idx]
    section_name = riasec_columns[int(end_idx/questions_per_section)-1]
    averages_df[section_name] = df[selected_cols].mean(axis=1) * (7/5)

final_df = pd.concat([df['Позначка часу'], averages_df], axis=1)
final_df_rm = final_df.where(final_df['Позначка часу'] == '13.12.2023 19:18:19')

final_df


,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04


In [464]:
df_merged = pd.merge(final_df, df, on='Позначка часу')

In [465]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [466]:
df_merged.fillna(0, inplace=True)

In [467]:
df_merged

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional,"Коли я отримую подарунки, я більше ціную практичні та корисні речі, ніж щось декоративне",Моя енергія ніколи не вичерпується,"Теоретичні ідеї та дилеми – це не те, на що варто витрачати багато часу\n(на скільки Ви погоджуєтесь з твердженням)",...,Я чудово вмію змусити інших підтримати мої ідеї\n\n,Я легко можу змінити думки оточуючих щодо якоїсь теми,Чи вам подобається приймати важливі рішення та вирішувати проблеми?,Чи берете ви участь у лідерських або підприємницьких ініціативах?,Чи вам подобається порядок і систематичність в роботі?,"Я завжди докладаю максимальних зусиль, щоб уникнути помилок\n\n",Чи легше вам працювати за чіткими правилами і встановленими стандартами?,Чи цікавитеся ви організаційними аспектами роботи?,"Я віддаю перевагу дотриманню правил, ніж імпровізації",Я легко концентруюсь на задачах які виконую та мене важко відволікти після погруження в роботу
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32,3,2,4,...,4,3,4,3,3,4,4,3,5,3
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92,2,3,1,...,3,2,3,2,2,3,2,4,3,4
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32,3,4,5,...,4,3,5,2,5,5,3,5,1,2
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96,1,1,5,...,5,2,1,1,1,1,1,3,1,1
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76,5,5,3,...,5,5,5,5,4,4,3,4,2,1
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48,1,1,1,...,1,1,4,1,4,2,3,4,3,3
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32,5,4,5,...,3,2,5,4,5,4,3,4,3,4
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20,5,2,3,...,2,3,3,1,2,3,5,2,3,4
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60,4,3,2,...,4,4,3,4,5,4,3,5,3,3
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04,4,2,4,...,2,3,4,3,4,3,4,3,4,3


In [468]:
X = df_merged.iloc[0:1, 7:43].values 
y = df_merged.iloc[0:1, 1:7].values
X

array([[3., 2., 4., 3., 5., 4., 3., 5., 4., 3., 3., 4., 3., 2., 4., 3.,
        4., 2., 3., 2., 3., 2., 3., 4., 3., 2., 4., 3., 4., 3., 3., 4.,
        4., 3., 5., 3.]])

In [469]:
y

array([[4.76, 5.04, 4.48, 3.64, 4.48, 5.32]])

In [470]:
X = df_merged.iloc[:, 7:37].values  
y = df_merged.iloc[:, 1:7].values

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

class FFNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(FFNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = FFNModel(input_size=X.shape[1], output_size=y.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
total_loss = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, targets)
        total_loss += loss.item()

average_loss = total_loss / len(test_loader)
print(f'Average Loss on Test Data: {average_loss}')

Epoch 1/250, Loss: 34.39323425292969
Epoch 2/250, Loss: 32.95744323730469
Epoch 3/250, Loss: 31.568315505981445
Epoch 4/250, Loss: 30.193965911865234
Epoch 5/250, Loss: 28.857717514038086
Epoch 6/250, Loss: 27.5463924407959
Epoch 7/250, Loss: 26.25442886352539
Epoch 8/250, Loss: 24.99142837524414
Epoch 9/250, Loss: 23.734825134277344
Epoch 10/250, Loss: 22.473722457885742
Epoch 11/250, Loss: 21.20466423034668
Epoch 12/250, Loss: 19.937292098999023
Epoch 13/250, Loss: 18.677383422851562
Epoch 14/250, Loss: 17.42734718322754
Epoch 15/250, Loss: 16.183717727661133
Epoch 16/250, Loss: 14.956912994384766
Epoch 17/250, Loss: 13.757132530212402
Epoch 18/250, Loss: 12.5881929397583
Epoch 19/250, Loss: 11.452458381652832
Epoch 20/250, Loss: 10.353029251098633
Epoch 21/250, Loss: 9.293489456176758
Epoch 22/250, Loss: 8.278435707092285
Epoch 23/250, Loss: 7.315495014190674
Epoch 24/250, Loss: 6.412109375
Epoch 25/250, Loss: 5.574273586273193
Epoch 26/250, Loss: 4.80506706237793
Epoch 27/250, Loss

In [471]:

df.fillna(0, inplace=True)



input_tensor = torch.tensor(X, dtype=torch.float32)

if len(input_tensor.shape) == 1:
    input_tensor = input_tensor.unsqueeze(0)

with torch.no_grad():
    predictions = model(input_tensor)

columns = ['R', 'I', 'A', 'S', 'E', 'C']
df_predictions = pd.DataFrame(predictions.numpy(), columns=columns)

df_predictions.insert(0, 'Позначка часу', df['Позначка часу'])
df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))
df_predictions

C:\Users\yg_ya\AppData\Local\Temp\ipykernel_2432\141191465.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_predictions.iloc[:, 1:] = df_predictions.iloc[:, 1:].applymap(lambda x: min(x, 7))


,Позначка часу,R,I,A,S,E,C
0,13.12.2023 17:14:56,4.340273,4.927232,4.193195,4.176572,4.905658,4.051835
1,13.12.2023 17:21:03,3.473434,3.977992,3.841079,4.038763,4.487669,3.813293
2,13.12.2023 17:47:13,5.945940,6.341672,5.920099,5.934582,5.815554,5.874948
3,13.12.2023 18:03:34,2.824297,2.238203,5.013378,2.813002,4.255398,2.707438
4,13.12.2023 18:05:36,5.557335,7.000000,5.488345,5.495428,6.437456,5.709637
5,13.12.2023 18:09:50,3.645844,4.285386,5.970172,3.771037,4.821690,4.308780
6,13.12.2023 18:42:14,5.431585,5.779009,4.282990,4.472754,5.482682,5.282914
7,13.12.2023 19:18:19,4.015495,4.801642,3.103567,3.920967,3.783521,3.970111
8,13.12.2023 21:09:46,5.234209,6.448050,6.180476,5.415363,6.046978,5.831924
9,16.12.2023 11:36:37,4.220679,4.448320,3.891061,4.446212,4.572609,4.175977


In [472]:
final_df

,Позначка часу,Realistic,Investigative,Artistic,Social,Enterprising,Conventional
0,13.12.2023 17:14:56,4.76,5.04,4.48,3.64,4.48,5.32
1,13.12.2023 17:21:03,3.64,3.92,3.92,3.64,3.64,3.92
2,13.12.2023 17:47:13,5.88,6.44,6.65,7.00,5.60,5.32
3,13.12.2023 18:03:34,2.52,1.40,5.88,2.24,4.20,1.96
4,13.12.2023 18:05:36,5.88,7.00,5.04,4.76,7.00,4.76
5,13.12.2023 18:09:50,3.36,4.76,7.00,3.64,3.36,4.48
6,13.12.2023 18:42:14,6.44,5.88,3.64,3.64,4.76,5.32
7,13.12.2023 19:18:19,4.76,5.04,2.24,4.48,3.36,4.20
8,13.12.2023 21:09:46,5.04,6.16,6.44,5.88,5.88,5.60
9,16.12.2023 11:36:37,4.20,3.92,3.92,4.48,4.48,5.04


In [473]:
input_score = [1, 2, 3, 4, 5, 6]
given_score = [0, 0, 0, 0, 0, 0]

def calculate_average_similarity(input_score, given_score):
    differences = [abs(a - b) / max(a, b) for a, b in zip(input_score, given_score)]
    average_difference = sum(differences) / len(differences)
    return 1 - average_difference

result = calculate_average_similarity(input_score, given_score)
print("Average Similarity:", result)


Average Similarity: 0.0


In [474]:
def similarity(init_data, external_data, k=2):
    for i in range(1,4):
        if int(external_data[-i]) == 0:
            continue
        init_data[int(external_data[-i]-1)]*=k
        external_data[int(external_data[-i]-1)]*=k
    prod=0
    mod_1=0
    mod_2=0
    for i in range(6):
        prod+=init_data[i]*external_data[i]
        mod_1+=init_data[i]**2
        mod_2+=external_data[i]**2
    return float(prod)/((mod_1**0.5)*(mod_2**0.5))




In [475]:
#GETTING N (by defaul 20) top matches:

def get_top_results(data, n=20, value_index=1):
    top_results = sorted(data, key=lambda x: x[value_index], reverse=True)[:n]
    return top_results

In [476]:
riasec_columns = ['R', 'I', 'A', 'S', 'E', 'C']

riasec_tuples = df_predictions[riasec_columns].apply(tuple, axis=1).tolist()

print(riasec_tuples[len(riasec_tuples)-1])

(4.4680585861206055, 5.612154960632324, 5.970699787139893, 4.207604885101318, 5.407166004180908, 5.010131359100342)


In [477]:
init_data = riasec_tuples[len(riasec_tuples)-1]

some_other_data_df = pd.read_excel('Interests.xlsx')

result = []

len(some_other_data_df)

for i in range (0, len(some_other_data_df ), 9):
    profession = some_other_data_df.loc[i, 'Title']

    R_score = some_other_data_df.loc[i, 'Data Value']
    I_score = some_other_data_df.loc[i+1, 'Data Value']
    A_score = some_other_data_df.loc[i+2, 'Data Value']
    S_score = some_other_data_df.loc[i+3, 'Data Value']
    E_score = some_other_data_df.loc[i+4, 'Data Value']
    C_score = some_other_data_df.loc[i+5, 'Data Value']
    HP_1 = some_other_data_df.loc[i+6, 'Data Value']
    HP_2 = some_other_data_df.loc[i+7, 'Data Value']
    HP_3 = some_other_data_df.loc[i+8, 'Data Value']

    row_values = (R_score, I_score, A_score, S_score, E_score, C_score, HP_1, HP_2, HP_3)



    turple = (profession, similarity(list(init_data), list(row_values)))
    result.append(turple)

print(get_top_results(result))

[('Instructional Designers and Technologists', 0.9845793115638246), ('Landscape Architects', 0.9789024694933642), ('Web Developers', 0.9760974317583154), ('Database Architects', 0.9759979817270078), ('Reporters and Correspondents', 0.9752918172142637), ('Curators', 0.9745265372325084), ('Green Marketers', 0.97418206350112), ('Architects, Except Landscape and Naval', 0.9735026794603219), ('Computer Network Architects', 0.9729416560427211), ('Intelligence Analysts', 0.9725108578791527), ('Cooks, Private Household', 0.9723576119997589), ('Video Game Designers', 0.9721629589631235), ('Hairdressers, Hairstylists, and Cosmetologists', 0.9719234736148656), ('Architectural Drafters', 0.9710755338308821), ('Operations Research Analysts', 0.9707335701997093), ('Ophthalmic Medical Technologists', 0.9706785771595015), ('Desktop Publishers', 0.9694853283654831), ('Anthropologists', 0.9692193444891946), ('Broadcast News Analysts', 0.9691197623570049), ('Sustainability Specialists', 0.968276348546500